In [1]:
!pip list

Package                  Version
------------------------ ---------
absl-py                  1.2.0
appdirs                  1.4.4
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
astor                    0.8.1
astunparse               1.6.3
attrs                    21.4.0
audioread                3.0.0
backcall                 0.2.0
beautifulsoup4           4.11.1
bleach                   4.1.0
cachetools               5.2.0
certifi                  2022.9.14
cffi                     1.15.1
charset-normalizer       2.1.1
click                    8.1.3
colorama                 0.4.5
cycler                   0.11.0
debugpy                  1.5.1
decorator                5.1.1
defusedxml               0.7.1
entrypoints              0.4
fastjsonschema           2.16.2
filelock                 3.8.0
gast                     0.3.3
google-auth              2.12.0
google-auth-oauthlib     0.4.6
google-pasta             0.2.0
grpcio                   1.49.1
h5py                  

In [1]:
# gpu 할당 확인
import tensorflow as tf

tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
tf.__version__

'2.3.0'

In [3]:
# 이 아래에서부터 모든 셀 gpu로 할당
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [4]:
### General imports ###
import os
from glob import glob
import pickle
import numpy as np

### Plot imports ###
from IPython.display import Image
import matplotlib.pyplot as plt

### Time Distributed ConvNet imports ###
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, TimeDistributed, concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, LeakyReLU, Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import backend as K
from keras.utils import np_utils
from keras.utils import plot_model
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import tensorflow as tf
from transformers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Warning ###
import warnings
warnings.filterwarnings('ignore')

In [6]:
data = np.load('./speechEmotion1.npz')
X_train=data['X_train']
y_train=data['y_train']
y_test=data['y_test']
X_test=data['X_test']

In [7]:
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(np.ravel(y_train)))


In [8]:
y_test = np_utils.to_categorical(lb.fit_transform(np.ravel(y_test)))

In [9]:
print(X_train.shape)
print(y_train.shape)

(453965, 180)
(453965, 7)


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_train,
                                           y_train,random_state=42 ,test_size=0.25)
                                                             # 랜덤 고정 숫자

In [12]:
# Reshape for convolution
X_train = X_train.reshape(X_train.shape[0], 3, 6, 10,1)
X_test = X_test.reshape(X_test.shape[0], 3, 6, 10,1)

# cnn(SER)

In [16]:
# Define two sets of inputs: MFCC and FBANK
input_y = Input(shape=X_train.shape[1:])

## First LFLB (local feature learning block)
y = Conv2D(16, kernel_size=(3, 3), strides=(1, 1), padding='same')(input_y)
y = BatchNormalization()(y)
y = Activation('elu')(y)
y = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(y)
y = Dropout(0.2)(y)     

## Second LFLB (local feature learning block)
y = Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('elu')(y)
y = MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same')(y)
y = Dropout(0.2)(y)

## Second LFLB (local feature learning block)
y = Conv2D(16, kernel_size=(3, 3), strides=(1, 1), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('elu')(y)
y = MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same')(y)
y = Dropout(0.2)(y)

## Second LFLB (local feature learning block)
y = Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('elu')(y)
y = MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same')(y)
y = Dropout(0.2)(y)  

## Flat
y = Flatten()(y)                      
                               
# Apply 2 LSTM layer and one FC

y = Dense(y_train.shape[1], activation='softmax', name='FC')(y)

# Build final model
model = Model(inputs=input_y, outputs=y)


ValueError: Input 0 of layer max_pooling2d_4 is incompatible with the layer: expected ndim=4, found ndim=5. Full shape received: [None, 3, 6, 10, 64]

In [35]:
# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Save best model
# best_model_save = ModelCheckpoint('./[CNN-LSTM]Model.hdf5', save_best_only=True, monitor='val_acc', mode='max')

# # Early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, verbose=1, mode='max')

# Fit model
history = model.fit(X_train, y_train, batch_size=64, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/100
5320/5320 [==============================] - 15s 3ms/step - loss: 0.9154 - accuracy: 0.6631 - val_loss: 0.6756 - val_accuracy: 0.7603
Epoch 2/100
5320/5320 [==============================] - 15s 3ms/step - loss: 0.8957 - accuracy: 0.6715 - val_loss: 0.6749 - val_accuracy: 0.7587
Epoch 3/100
5320/5320 [==============================] - 15s 3ms/step - loss: 0.8769 - accuracy: 0.6784 - val_loss: 0.6416 - val_accuracy: 0.7762
Epoch 4/100
5320/5320 [==============================] - 15s 3ms/step - loss: 0.8631 - accuracy: 0.6834 - val_loss: 0.6182 - val_accuracy: 0.7845
Epoch 5/100
5320/5320 [==============================] - 15s 3ms/step - loss: 0.8516 - accuracy: 0.6889 - val_loss: 0.6101 - val_accuracy: 0.7875
Epoch 6/100
5320/5320 [==============================] - 15s 3ms/step - loss: 0.8430 - accuracy: 0.6915 - val_loss: 0.5847 - val_accuracy: 0.7963
Epoch 7/100
5320/5320 [==============================] - 15s 3ms/step - loss: 0.8328 - accuracy: 0.6946 - val_loss: 0.5913 -

Epoch 57/100
5320/5320 [==============================] - 15s 3ms/step - loss: 0.7136 - accuracy: 0.7411 - val_loss: 0.4431 - val_accuracy: 0.8471
Epoch 58/100
5320/5320 [==============================] - 15s 3ms/step - loss: 0.7140 - accuracy: 0.7413 - val_loss: 0.4577 - val_accuracy: 0.8413
Epoch 59/100
5320/5320 [==============================] - 15s 3ms/step - loss: 0.7127 - accuracy: 0.7420 - val_loss: 0.4595 - val_accuracy: 0.8414
Epoch 60/100
5320/5320 [==============================] - 15s 3ms/step - loss: 0.7106 - accuracy: 0.7434 - val_loss: 0.4538 - val_accuracy: 0.8436
Epoch 61/100
5320/5320 [==============================] - 15s 3ms/step - loss: 0.7118 - accuracy: 0.7419 - val_loss: 0.4401 - val_accuracy: 0.8482
Epoch 62/100
5320/5320 [==============================] - 15s 3ms/step - loss: 0.7097 - accuracy: 0.7422 - val_loss: 0.4441 - val_accuracy: 0.8466
Epoch 63/100
5320/5320 [==============================] - 15s 3ms/step - loss: 0.7082 - accuracy: 0.7432 - val_loss: 0

In [36]:
model.save('voice_emotion_CnnModel1008.h5')

In [47]:

# Define two sets of inputs: MFCC and FBANK
input_y = Input(shape=X_train.shape[1:])

## First LFLB (local feature learning block)
y = TimeDistributed(Conv2D(16, kernel_size=(3, 3), strides=(1, 1), padding='same'))(input_y)
y = TimeDistributed(BatchNormalization())(y)
y = TimeDistributed(Activation('elu'))(y)
y = TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))(y)
y = TimeDistributed(Dropout(0.2))(y)     

## Second LFLB (local feature learning block)
y = TimeDistributed(Conv2D(16, kernel_size=(3, 3), strides=(1, 1), padding='same'))(y)
y = TimeDistributed(BatchNormalization())(y)
y = TimeDistributed(Activation('elu'))(y)
y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))(y)
y = TimeDistributed(Dropout(0.2))(y)

## Second LFLB (local feature learning block)
y = TimeDistributed(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same'))(y)
y = TimeDistributed(BatchNormalization())(y)
y = TimeDistributed(Activation('elu'))(y)
y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))(y)
y = TimeDistributed(Dropout(0.2))(y)

## Second LFLB (local feature learning block)
y = TimeDistributed(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same'))(y)
y = TimeDistributed(BatchNormalization())(y)
y = TimeDistributed(Activation('elu'))(y)
y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))(y)
y = TimeDistributed(Dropout(0.2))(y)  

## Flat
y = TimeDistributed(Flatten())(y)                      
                               
# Apply 2 LSTM layer and one FC
y = LSTM(64, return_sequences=False, dropout=0.2)(y)
y = Dense(32, activation='relu')(y)
y = Dense(16, activation='relu')(y)
y = Dense(y_train.shape[1], activation='softmax')(y)

# Build final model
CN_LS_model = Model(inputs=input_y, outputs=y)


In [48]:
# Compile model
CN_LS_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Save best model
# best_model_save = ModelCheckpoint('./[CNN-LSTM]Model.hdf5', save_best_only=True, monitor='val_acc', mode='max')

# # Early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, verbose=1, mode='max')

# Fit model
history = CN_LS_model.fit(X_train, y_train, batch_size=64, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/100
5320/5320 [==============================] - 22s 4ms/step - loss: 1.6594 - accuracy: 0.3287 - val_loss: 1.4811 - val_accuracy: 0.4219
Epoch 2/100
5320/5320 [==============================] - 21s 4ms/step - loss: 1.5233 - accuracy: 0.4068 - val_loss: 1.3629 - val_accuracy: 0.4821
Epoch 3/100
5320/5320 [==============================] - 21s 4ms/step - loss: 1.4784 - accuracy: 0.4308 - val_loss: 1.3214 - val_accuracy: 0.5045
Epoch 4/100
5320/5320 [==============================] - 21s 4ms/step - loss: 1.4551 - accuracy: 0.4404 - val_loss: 1.2813 - val_accuracy: 0.5199
Epoch 5/100
5320/5320 [==============================] - 21s 4ms/step - loss: 1.4384 - accuracy: 0.4491 - val_loss: 1.2666 - val_accuracy: 0.5272
Epoch 6/100
5320/5320 [==============================] - 21s 4ms/step - loss: 1.4290 - accuracy: 0.4535 - val_loss: 1.2522 - val_accuracy: 0.5351
Epoch 7/100
5320/5320 [==============================] - 21s 4ms/step - loss: 1.4196 - accuracy: 0.4570 - val_loss: 1.2272 -

Epoch 57/100
5320/5320 [==============================] - 20s 4ms/step - loss: 1.3321 - accuracy: 0.4934 - val_loss: 1.1058 - val_accuracy: 0.6011
Epoch 58/100
5320/5320 [==============================] - 20s 4ms/step - loss: 1.3354 - accuracy: 0.4917 - val_loss: 1.1137 - val_accuracy: 0.5944
Epoch 59/100
5320/5320 [==============================] - 20s 4ms/step - loss: 1.3327 - accuracy: 0.4926 - val_loss: 1.1057 - val_accuracy: 0.5989
Epoch 60/100
5320/5320 [==============================] - 20s 4ms/step - loss: 1.3337 - accuracy: 0.4929 - val_loss: 1.0998 - val_accuracy: 0.5997
Epoch 61/100
5320/5320 [==============================] - 20s 4ms/step - loss: 1.3315 - accuracy: 0.4932 - val_loss: 1.0939 - val_accuracy: 0.6035
Epoch 62/100
5320/5320 [==============================] - 20s 4ms/step - loss: 1.3300 - accuracy: 0.4948 - val_loss: 1.0996 - val_accuracy: 0.5958
Epoch 63/100
5320/5320 [==============================] - 20s 4ms/step - loss: 1.3307 - accuracy: 0.4944 - val_loss: 1

In [15]:

CN_LS_model.save('voice_emotion_CnnLstmModel1008.h5')

In [1]:
from tensorflow.keras.models import load_model

In [2]:
cnn=load_model('./voice_emotion_CnnModel1008.h5')

In [4]:
cnn.inputs

[<tf.Tensor 'Input_MELSPECT:0' shape=(None, 30, 6, 1) dtype=float32>]

In [30]:
cnn.evaluate(X_test,y_test)

1758/1758 [==============================] - 4s 2ms/step - loss: 3.7741 - accuracy: 0.1228


[3.774106025695801, 0.12284880876541138]